## Deliverable 2. Create a Customer Travel Destinations Map.

In [19]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [20]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:\\Users\\lag86\\OneDrive\\Desktop\\BOOTCAMP STUFF\\Asynchronous---Challenge\\Module 6 - APIs\\World_Weather_Analysis- clone\\World_Weather_Analysis\\Weather_Database\\WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Eureka,US,40.8021,-124.1637,57.67,88,88,10.00,overcast clouds
1,1,Mataura,NZ,-46.1927,168.8643,62.20,73,8,1.45,clear sky
2,2,Pavino,RU,59.1139,46.1414,3.96,100,90,3.74,overcast clouds
3,3,San Juan,AR,-31.5375,-68.5364,83.43,18,0,29.53,clear sky
4,4,Pangnirtung,CA,66.1451,-65.7125,19.40,79,100,3.44,overcast clouds


In [23]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [24]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,San Juan,AR,-31.5375,-68.5364,83.43,18,0,29.53,clear sky
8,8,Hithadhoo,MV,-0.6000,73.0833,81.64,68,7,6.08,clear sky
10,10,Jacareacanga,BR,-6.2222,-57.7528,75.88,97,82,2.53,moderate rain
13,13,Vaini,TO,-21.2000,-175.2000,82.56,74,20,5.75,few clouds
18,18,Malakal,SS,9.5334,31.6605,86.07,16,10,4.92,clear sky
19,19,Chuy,UY,-33.6971,-53.4616,75.16,59,6,5.14,moderate rain
21,21,San Felipe,VE,10.3399,-68.7425,76.15,83,97,8.23,overcast clouds
23,23,Kendari,ID,-3.9450,122.4989,75.45,93,100,3.24,overcast clouds
29,29,Butaritari,KI,3.0707,172.7902,81.01,70,100,12.84,overcast clouds
32,32,Miandrivazo,MG,-19.5167,45.4667,75.00,87,96,3.47,overcast clouds


In [25]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             216
City                   216
Country                216
Lat                    216
Lng                    216
Max Temp               216
Humidity               216
Cloudiness             216
Wind Speed             216
Current Description    216
dtype: int64

In [26]:
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [27]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
try:
    clean_df = preferred_cities_df.dropna()
    
except:
    print('No empty rows to drop.')
    clean_df = preferred_cities_df

In [28]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,San Juan,AR,83.43,clear sky,-31.5375,-68.5364,
8,Hithadhoo,MV,81.64,clear sky,-0.6000,73.0833,
10,Jacareacanga,BR,75.88,moderate rain,-6.2222,-57.7528,
13,Vaini,TO,82.56,few clouds,-21.2000,-175.2000,
18,Malakal,SS,86.07,clear sky,9.5334,31.6605,
19,Chuy,UY,75.16,moderate rain,-33.6971,-53.4616,
21,San Felipe,VE,76.15,overcast clouds,10.3399,-68.7425,
23,Kendari,ID,75.45,overcast clouds,-3.9450,122.4989,
29,Butaritari,KI,81.01,overcast clouds,3.0707,172.7902,
32,Miandrivazo,MG,75.00,overcast clouds,-19.5167,45.4667,


In [36]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")
        hotel_df.loc[index,"Hotel Name"] = np.nan
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [37]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,San Juan,AR,83.43,clear sky,-31.5375,-68.5364,Hotel Provincial
8,Hithadhoo,MV,81.64,clear sky,-0.6000,73.0833,Scoop Guest House
10,Jacareacanga,BR,75.88,moderate rain,-6.2222,-57.7528,Casa do Ian😎
13,Vaini,TO,82.56,few clouds,-21.2000,-175.2000,Keleti Beach Resort
18,Malakal,SS,86.07,clear sky,9.5334,31.6605,Nile Palace Hotel


In [38]:
# 7. Drop the rows where there is no Hotel Name.
try:
    clean_hotel_df = hotel_df.dropna()
    
except:
    print("No empty rows to drop.")
    clean_hotel_df

In [41]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,San Juan,AR,83.43,clear sky,-31.5375,-68.5364,Hotel Provincial
8,Hithadhoo,MV,81.64,clear sky,-0.6000,73.0833,Scoop Guest House
10,Jacareacanga,BR,75.88,moderate rain,-6.2222,-57.7528,Casa do Ian😎
13,Vaini,TO,82.56,few clouds,-21.2000,-175.2000,Keleti Beach Resort
18,Malakal,SS,86.07,clear sky,9.5334,31.6605,Nile Palace Hotel
...,...,...,...,...,...,...,...
708,Ban Phai,TH,75.42,clear sky,16.0600,102.7310,Inpawa Hotel
710,Bo Rai,TH,80.76,broken clouds,12.5728,102.5371,Baiboon Resort
714,Parana,AR,76.55,broken clouds,-31.7320,-60.5238,Howard Johnson by Wyndham Plaza Hotel Mayorazgo
716,Saint-Joseph,RE,75.58,moderate rain,-21.3667,55.6167,"""Plantation Bed and Breakfast"


In [47]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [48]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [49]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))